In [ ]:
%load_ext autoreload
%autoreload 2

import docker
import git
import os
import subprocess
import yaml

from subprocess import PIPE, STDOUT

from defaults import motion_list, regressor_list
from subjects import generate_subject_list_for_range

with open('../configs/cpac.yml', 'r') as config_file:
    config_settings = yaml.safe_load(config_file)
    
cpac_dir = os.path.abspath(os.path.join(os.getcwd(), *[os.pardir for i in range(3)]))    
n_cpus = 10
pipeline_file = os.path.join(cpac_dir, 'dev/docker_data/default_pipeline.yml')
tag = git.Repo(search_parent_directories=True).head.object.hexsha[:7]
config_settings['this_run_outputs'] = os.path.abspath(f'~/outputs/{tag}')
try:
    os.makedirs(config_settings['this_run_outputs'], exist_ok=True)
except PermissionError:
    config_settings['this_run_outputs'] = os.path.abspath(f'~/outputs/{tag}')
    os.makedirs(config_settings['this_run_outputs'], exist_ok=True)
client = docker.from_env()
client.images.build(path=cpac_dir, tag=f"fcpindi/c-pac:{tag}")

In [ ]:
client = docker.from_env()
client.images.build(path=cpac_dir, tag=f"fcpindi/c-pac:{tag}")

In [ ]:
subprocess.run(
    " ".join([
        f"docker run -v {config_settings['this_run_outputs']}:/outputs "
        f"-v {config_settings['data_path']}:/bids_dataset:ro "
        f"-v /data3/cnl/fmriprep/cpac_run/configs:/configs fcpindi/c-pac:{tag} "
        f"/bids_dataset /outputs participant --pipeline_file {pipeline_file} "
        f"--n_cpus {n_cpus} --save_working_dir --participant_label",
        *generate_subject_list_for_range(
            (config_settings['subjects']['start'], config_settings['subjects']['stop'])
        )
    ]).split(" "),
    stdout=PIPE,
    stderr=STDOUT
)